In [1]:
import geopandas as gpd
import rasterio
from rasterio import features
import pandas as pd
import numpy as np
import os
from shapely.geometry import LineString

os.chdir('/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/Data')

In [4]:
import geopandas as gpd
import rasterio
import numpy as np
import pandas as pd
from shapely.geometry import LineString
from pyproj import CRS

# Load the flowlines GeoJSON file
# Replace 'path_to_flowlines.geojson' with your actual file path
flowlines = gpd.read_file('final_cleaned_gdf.geojson')

# Load the DEM GeoTIFF file
# Replace 'path_to_dem.tif' with your actual file path
dem = rasterio.open('output_USGS30m.tif')

# Ensure both datasets are in the same CRS
dem_crs = CRS(dem.crs)

if flowlines.crs != dem_crs:
    flowlines = flowlines.to_crs(dem_crs)

def sample_elevations_along_line(line_geom, dem, num_samples=100):
    """
    Samples elevations along a line geometry from a DEM raster.
    
    Parameters:
    - line_geom: Shapely LineString geometry of the flowline.
    - dem: Opened rasterio DEM object.
    - num_samples: Number of points to sample along the line.

    Returns:
    - elevations: List of elevation values sampled from the DEM.
    """
    # Generate equally spaced points along the line
    distances = np.linspace(0, line_geom.length, num_samples)
    points = [line_geom.interpolate(distance) for distance in distances]
    
    # Get the coordinates of the points
    coords = [(point.x, point.y) for point in points]
    
    # Sample the DEM at these points
    elevations = []
    for coord in coords:
        try:
            # Sample the DEM at the coordinate
            elevation = list(dem.sample([coord]))[0][0]
            if dem.nodata is not None and elevation == dem.nodata:
                elevations.append(None)
            else:
                elevations.append(elevation)
        except Exception:
            elevations.append(None)
    
    # Filter out None or NaN values
    elevations = [e for e in elevations if e is not None and not np.isnan(e)]
    return elevations

# Prepare a list to store the results
results = []

# Iterate over each flowline
for idx, row in flowlines.iterrows():
    line_geom = row.geometry
    
    # Sample elevations along the line
    elevations = sample_elevations_along_line(line_geom, dem)
    
    # Calculate statistics if we have valid elevations
    if elevations:
        max_elev = max(elevations)
        min_elev = min(elevations)
        avg_elev = sum(elevations) / len(elevations)
    else:
        max_elev = min_elev = avg_elev = None
    
    # Replace 'id' below with the actual ID column name in your GeoDataFrame
    results.append({
        'Flowline_ID': row['flowline_id'],  # <-- Use your actual ID field name here
        'Max_Elevation': max_elev,
        'Min_Elevation': min_elev,
        'Avg_Elevation': avg_elev
    })

# Create a DataFrame from the results
elevation_df = pd.DataFrame(results)

# Export the results to an Excel file
# Replace 'flowline_elevations.xlsx' with your desired output file name
elevation_df.to_excel('flowline_elevations.xlsx', index=False)

print("Elevation statistics calculated and saved to 'flowline_elevations.xlsx'.")

Elevation statistics calculated and saved to 'flowline_elevations.xlsx'.


In [5]:
elevation_df.head()

,Flowline_ID,Max_Elevation,Min_Elevation,Avg_Elevation
0,470450.0,1612.875122,1612.644043,1612.733154
1,477982.0,1419.916626,1419.780396,1419.848389
2,457300.0,1488.291382,1488.250488,1488.272827
3,465120.0,1482.529419,1482.448242,1482.481201
4,466186.0,1470.254272,1469.974487,1470.031616
